```{contents}
```
## **Fan-In in LangGraph**

**Fan-In** is a control-flow pattern in LangGraph where **multiple parallel execution paths converge into a single node** that aggregates their results.
It is the foundation for **parallel computation, multi-agent coordination, ensemble reasoning, and consensus building**.

---

### **1. Intuition**

When a task can be decomposed into independent sub-tasks, LangGraph allows them to run **in parallel (fan-out)** and later **merge their outputs (fan-in)**.

```
           Task A ─┐
                   ├──► Merge ──► Next Step
           Task B ─┘
```

Fan-in turns **distributed work** into a **single coherent state**.

---

### **2. Why Fan-In Is Critical**

| Requirement         | Role of Fan-In              |
| ------------------- | --------------------------- |
| Parallel speedup    | Combine independent results |
| Multi-agent systems | Merge agent outputs         |
| Reliability         | Cross-check & verify        |
| Reasoning quality   | Ensemble improvement        |
| Scalability         | Decompose large tasks       |

---

### **3. How Fan-In Works in LangGraph**

LangGraph does **not** introduce a special “fan-in” node.
Fan-in is implemented by:

1. **Parallel edges** → multiple nodes update shared state
2. **Reducers** → deterministic state merging
3. **Join node** → reads combined state

---

### **4. State Design for Fan-In**

Shared state must support aggregation.

```python
from typing import TypedDict, List

class State(TypedDict):
    results: List[str]
```

Reducer:

```python
def merge_results(existing: List[str], new: List[str]) -> List[str]:
    return existing + new
```

---

### **5. Minimal Working Example**

```python
from langgraph.graph import StateGraph, END
from typing import TypedDict, List

class State(TypedDict):
    results: List[str]

def task_a(state):
    return {"results": ["A done"]}

def task_b(state):
    return {"results": ["B done"]}

def merge(state):
    return {"results": state["results"]}

builder = StateGraph(State)

builder.add_node("A", task_a)
builder.add_node("B", task_b)
builder.add_node("merge", merge)

builder.set_entry_point("A")
builder.add_edge("A", "merge")
builder.add_edge("B", "merge")

graph = builder.compile()

output = graph.invoke({"results": []})
print(output)
```

This graph conceptually executes:

```
A ─┐
    ├──► merge ──► END
B ─┘
```

---

### **6. Fan-In with Conditional Synchronization**

Often you want to wait for **N agents**.

```python
class State(TypedDict):
    responses: dict
    count: int
```

Each agent increments `count`.
The join node proceeds when `count == N`.

---

### **7. Common Fan-In Patterns**

| Pattern               | Description                      |
| --------------------- | -------------------------------- |
| Map-Reduce            | Parallel compute → aggregate     |
| Multi-Agent Consensus | Independent opinions → agreement |
| Ensemble LLM          | Multiple models → best answer    |
| Verification Pipeline | Generate → check → merge         |
| Data Processing       | Shard → compute → combine        |

---

### **8. Production Considerations**

| Concern         | Solution                        |
| --------------- | ------------------------------- |
| Race conditions | Deterministic reducers          |
| Partial failure | Fallback nodes                  |
| Latency         | Timeout & speculative execution |
| Memory growth   | Prune merged state              |
| Debugging       | Trace each branch               |

---

### **9. Mental Model**

> **Fan-out creates intelligence in parallel.
> Fan-in converts it into a single decision.**

This pattern enables LangGraph to function as a **distributed reasoning engine** rather than a linear pipeline.



### Demonstration

        Worker A ─┐
                  ├──► Join / Fan-In ──► Final
        Worker B ─┤
                  │
        Worker C ─┘


In [14]:
from typing import TypedDict, Dict
from langgraph.graph import add_messages
from typing_extensions import Annotated


from typing import TypedDict, Dict
from typing_extensions import Annotated

def merge_results(old: Dict[str, str], new: Dict[str, str]):
    return {**old, **new}

def sum_completed(old: int, new: int):
    return old + new

class State(TypedDict):
    results: Annotated[Dict[str, str], merge_results]
    completed: Annotated[int, sum_completed]


def worker_a(state):
    return {"results": {"A": "Data from A"}, "completed": 1}

def worker_b(state):
    return {"results": {"B": "Data from B"}, "completed": 1}

def worker_c(state):
    return {"results": {"C": "Data from C"}, "completed": 1}

def join_router(state):
    if state["completed"] >= 3:
        return "final"
    return END


def final_node(state):
    summary = " | ".join(state["results"].values())
    return {"results": {"final": summary}}


In [15]:
from langgraph.graph import StateGraph, END

builder = StateGraph(State)

builder.add_node("A", worker_a)
builder.add_node("B", worker_b)
builder.add_node("C", worker_c)
builder.add_node("final", final_node)

builder.set_entry_point("A")

builder.add_edge("A", "B")
builder.add_edge("B", "C")

builder.add_conditional_edges("C", join_router, {
    "final": "final",
    END: END
})

builder.add_edge("final", END)

graph = builder.compile()

In [16]:
output = graph.invoke({"results": {}, "completed": 0})
print(output)


{'results': {'A': 'Data from A', 'B': 'Data from B', 'C': 'Data from C', 'final': 'Data from A | Data from B | Data from C'}, 'completed': 3}
